<img width=800 src="http://robotics.eecs.berkeley.edu/~ronf/biomimetics-thin.jpg">

# Robot Test

In [1]:
import time
from python.mbedrpc import *
import threading

In [2]:
class Motor:
    def __init__(self, a1, a2):
        self.a1=a1
        self.a2=a2
        
    def cmd(self, speed):
        if speed >=0:
            self.a1.write(speed)
            self.a2.write(0)
        else:
            self.a1.write(0)
            self.a2.write(-speed)
class Robot:
    def __init__(self, dev='/dev/ttyACM0'):
        self.mbed=SerialRPC(dev, 115200)
        a1=PwmOut(self.mbed, p21)
        a2=PwmOut(self.mbed, p22)
        b1=PwmOut(self.mbed, p23)
        b2=PwmOut(self.mbed, p24)
        self.led1 = DigitalOut(self.mbed, LED1)
        self.led2 = DigitalOut(self.mbed, LED2)
        self.led3 = DigitalOut(self.mbed, LED3)
        self.led4 = DigitalOut(self.mbed, LED4)
        
        self.m_right = Motor(a1, a2)
        self.m_left = Motor(b1, b2)
        self.enabled=True
        self.last_left=0
        self.last_right=0
        self.sensors=[]
        for i in (p20,p19,p18,p17,p16,p15):
            self.sensors.append(AnalogIn(self.mbed, i))
        self.rlock=threading.Lock()
    def enable(self):
        self.rlock.acquire()
        self.enabled=True
        self._cmd(self.last_left, self.last_right)
        self.rlock.release()
    def disable(self):
        self.rlock.acquire()
        self.enabled=False
        self._cmd(self.last_left, self.last_right)
        self.rlock.release()
    def drive(self, left, right):
        self.rlock.acquire()
        self._cmd(left, right)
        self.rlock.release()
    def cmd(self, left, right):
        self.rlock.acquire()
        self._cmd(left, right)
        self.rlock.release()
    def _cmd(self, left, right):
        self.last_left=left
        self.last_right=right
        if self.enabled:
            self.m_left.cmd(-left)
            self.m_right.cmd(right)
        else:
            self.m_left.cmd(0)
            self.m_right.cmd(0)
    def read_sensors(self):
        """ returns an array of the line sensor reflectance values
        """
        self.rlock.acquire()
        def read(sensor): return sensor.read()
        retu=map(read, self.sensors)
        self.rlock.release()
        return retud
    def setLED1(self, state):
        self.led1.write(state)
    def setLED2(self, state):
        self.led2.write(state)
    def setLED3(self, state):
        self.led3.write(state)
    def setLED4(self, state):
        self.led4.write(state)

In [3]:
ls /dev/ttyACM*

/dev/ttyACM0  /dev/ttyACM99@


In [4]:
r0=Robot('/dev/ttyACM0')

In [16]:
delaytime = 0.03

for i in range(100):
    r0.setLED1(1)
    time.sleep(delaytime)
    r0.setLED2(1)
    time.sleep(delaytime)
    r0.setLED3(1)
    time.sleep(delaytime)
    r0.setLED4(1)
    time.sleep(delaytime)
    
    r0.setLED1(0)
    time.sleep(delaytime)
    r0.setLED2(0)
    time.sleep(delaytime)
    r0.setLED3(0)
    time.sleep(delaytime)
    r0.setLED4(0)
    time.sleep(delaytime)

In [15]:
r0.cmd(0.1,0.5)
time.sleep(1)
r0.cmd(0,0)

In [14]:
r0.cmd(1.0,1.0)
time.sleep(1)
r0.cmd(0,0)

Now, Restart the kernel to release ownership of the serial device:

File > Kernel > Restart